In [1]:
!pip install fastapi nest_asyncio pyngrok uvicorn

In [5]:
!pip install llmware

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 MB 38.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 84.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 104.5 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pymongo
    Found existing installation: pymongo 3.13.0
    Uninstalling pymongo-3.13.0:
      Successfully uninstalled pymongo-3.13.0
  Attempting uninstall: botocore
    Found existing installation: botocore 1.35.23
    Uninstalling botocore-1.35.23:
      Successfully uninstalled botocore-1.35.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.


In [6]:

""" This example illustrates how to use the slim-extract model to extract custom keys from selected text.
    We have included a set of sample earnings releases (comprising lines ~10 - ~385 of this script), and run a
    simple loop through the earnings releases, showing how to create an extract prompt to identify
    'revenue growth' from these examples.

    There are several function-calling models in the slim-extract family, fine-tuned on multiple leading
    small model base foundations - full list and options are below in the code.  """

from llmware.models import ModelCatalog

# Sample earnings releases

earnings_releases = [
    {"context": "I am a loco pilot in Indian Railways. On June 30, 2021, while driving train number 06405, the Parassuram Express from Kozhikode to Ernakulam, an accident occurred at Arakkinar, Kozhikode, Kerala. A woman and her child were struck by the train. Despite applying the brakes, the train came to a halt about 100 meters past the accident site. Tragically, the two were killed, and their bodies were scattered into pieces. I immediately informed the guard and the RPF. However, the horrific sight of their remains continues to haunt me."}

]

#   *** Execution script starts here ***

#   specialized function-calling extraction models on top of several leading small model bases,
#   ranging from 0.5b (qwen2) - 3.8b (phi3)

slim_extract_models = ["slim-extract-tool",                 #   original - stablelm-3b (2.7b)
                       "slim-extract-tiny-tool",            #   tiny-llama 1.1b
                       "slim-extract-qwen-1.5b-gguf",       #   **NEW** qwen 1.5b
                       "slim-extract-phi-3-gguf",           #   **NEW** phi-3 (3.8b)
                       "slim-extract-qwen-0.5b-gguf"]       #   **NEW** qwen 0.5b

#   load the model
model = ModelCatalog().load_model("slim-extract-tool",sample=False,temperature=0.0, max_output=100)

#   iterate through the earnings release samples above
for i, sample in enumerate(earnings_releases):

    print(sample["context"])
      #   key line: execute function_call on selected model with 'custom_key' = "revenue growth"
    response = model.function_call(sample["context"], function="extract", params=["date of accident", "Number of persons involed in accident"])

    #   display the response on the screen
    print("extract response: ", i, response["llm_response"])


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1204: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

slim-extract.gguf:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/38.7k [00:00<?, ?B/s]

LLMWareException: Models - load_model - pull_snapshot_from_hf - llmware/slim-extract-tool - Something has gone wrong in the download process.   Please try again.

In [7]:
from llmware.models import ModelCatalog

model = ModelCatalog().load_model("bartowski/Meta-Llama-3-8B-Instruct-GGUF")
text_passage = "I want an accident trauma councellor"


model3 = ModelCatalog().load_model("slim-extract-tool",sample=False,temperature=0.0, max_output=100)


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1204: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Meta-Llama-3-8B-Instruct-Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:   no
ggml_cuda_init: CUDA_USE_TENSOR_CORES: yes
ggml_cuda_init: found 2 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
  Device 1: Tesla T4, compute capability 7.5, VMM: yes


In [9]:
from fastapi import FastAPI, Body
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

app = FastAPI()

# Middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

# Model to validate incoming message
class Message(BaseModel):
    name: str
    text: str

# Global variables to store the extracted data
dateOp = None
placeOp = None
nameOp = None
chat_history = ""

@app.get('/')
async def root():
    return {'reply': "Hi, I am your personal accident trauma councellor. It's hard going through this situation. But beileve me you will be better."}

@app.post('/')
async def receive_message(message: Message = Body(...)):
    global latest_user_message
    global chat_history
    global dateOp, placeOp, nameOp

    # Store the received message
    latest_user_message = message
    print(latest_user_message.text)

    # Extract data (date, place, name) using model3
    try:
        date = model3.function_call(latest_user_message.text, function="extract", params=["date"])
        dateOp = date['llm_response']['date']
        print("Date:", dateOp)
    except Exception as e:
        dateOp = None
        print(f"Error extracting date: {e}")

    try:
        place = model3.function_call(latest_user_message.text, function="extract", params=["place"])
        placeOp = place['llm_response']['place']
        print("Place:", placeOp)
    except Exception as e:
        placeOp = None
        print(f"Error extracting place: {e}")

    try:
        name = model3.function_call(latest_user_message.text, function="extract", params=["name"])
        nameOp = name['llm_response']['name']
        print("Name:", nameOp)
    except Exception as e:
        nameOp = None
        print(f"Error extracting name: {e}")
    print(dateOp, nameOp, placeOp)
    # Get response from the LLM
    response = model.inference(latest_user_message.text, add_context=chat_history)
    print("Response:", response)
    llm_response = response['llm_response']

    # Add the query and response to the chat_history string
    query_response_pair = f"Query: {latest_user_message.text}\nResponse: {llm_response}\n"
    chat_history += query_response_pair

    # Return response along with the chat history
    return {
        'reply': llm_response,
        'status': 'success',
    }

@app.get("/extract_data")
async def get_extracted_data():
    """
    This route returns the extracted date, place, and name data as an array of arrays.
    """
    # Ensure the extracted data is available
    if not dateOp or not placeOp or not nameOp:
        return {"error": "Missing extracted data"}
    
    # Return the array of arrays containing date, place, and name
    return {
        'data': [
            dateOp,
            placeOp,
            nameOp
        ]
    }


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
auth_token = "2pUdtioux8gjENwj4vdh6lLZUcK_45Uu5dTaJ6dBq4YkHswWj"

# Set the authtoken
ngrok.set_auth_token(auth_token)

# Connect to ngrok
ngrok_tunnel = ngrok.connect(8000, hostname = 'grand-mackerel-urgently.ngrok-free.app')

# Print the public URL
print('Public URL:', ngrok_tunnel.public_url)

# Apply nest_asyncio
nest_asyncio.apply()

# Run the uvicorn server
uvicorn.run(app, port=8000)

INFO:     Started server process [29]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://grand-mackerel-urgently.ngrok-free.app
INFO:     111.92.80.102:0 - "GET / HTTP/1.1" 200 OK
INFO:     111.92.80.102:0 - "OPTIONS / HTTP/1.1" 200 OK
I am a loco pilot in Indian Railways. On June 30, 2021, while driving train number 06405, the Parassuram Express from Kozhikode to Ernakulam, an accident occurred at Arakkinar, Kozhikode, Kerala. A woman and her child were struck by the train. Despite applying the brakes, the train came to a halt about 100 meters past the accident site. Tragically, the two were killed, and their bodies were scattered into pieces.
Date: ['June 30, 2021']
Place: ['Kozhikode, Kerala']
Name: []
['June 30, 2021'] [] ['Kozhikode, Kerala']
Response: {'llm_response': "I'm so sorry to hear about the tragic accident that occurred on June 30, 2021, while you were driving the Parassuram Express (train number 06405) from Kozhikode to Ernakulam. Losing two innocent lives, including a woman and her child, must have been a devastating experience for you 